In [35]:
import string

import lyricsgenius
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
from adjustText import adjust_text
from nltk.corpus import stopwords

# API setup
# NOTE: this requires setup of Spotify and Genius developer accounts for credentials
genius = lyricsgenius.Genius('f-L9hQZOQ1VBDYt_lhkYUd-3wf0qk0fvXtwetdAWGQj-c9ICYmFx0bEwqaXNgb7cs8rtSphCVDvUUuH-rCEdKQ')

# Set up stop words and punctuation for removal
filterWords = stopwords.words("english")
translator = str.maketrans('', '', string.punctuation)

In [36]:
df_grammys = pd.read_csv('grammy_df.csv')

In [37]:
df_grammys

,title,artist,winner,year
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958
1,Catch A Falling Star,Perry Como,0,1958
2,Witchcraft (Single),Frank Sinatra,0,1958
3,The Chipmunk Song,David Seville And The Chipmunks,0,1958
4,Fever (Single),Peggy Lee,0,1958
...,...,...,...,...
341,Woman,Doja Cat,0,2022
342,You And Me On The Rock,Brandi Carlile Featuring Lucius,0,2022
343,Good Morning Gorgeous,Mary J. Blige,0,2022
344,Easy On Me,Adele,0,2022


In [38]:
def get_lyrics_from_dataframe(df, access_token, timeout=40):
    # Initialize Genius API client with timeout
    genius = lyricsgenius.Genius(access_token, timeout=timeout)

    # Iterate over rows in dataframe
    lyrics = []
    song_names = []
    for index, row in df.iterrows():
        song_title = row['title']
        song = genius.search_song(song_title)
        if song is not None:
            song_lyrics = song.lyrics
            lyrics.append(song_lyrics)
            song_names.append(song_title)

    # Create dataframe with lyrics
    lyrics_df = pd.DataFrame(data=lyrics, index=song_names, columns=['lyrics'])
    return lyrics_df



In [39]:
df_lyrics = get_lyrics_from_dataframe(df_grammys,'ulQdQcjIRwX5BBh3pnUsTznTZpXIMbcbhWsiKhw37BdmAlkCLlfX0emCVpPSvn3Z')

Searching for "Nel Blu Dipinto Di Blu (Volare)"...
Done.
Searching for "Catch A Falling Star"...
Done.
Searching for "Witchcraft (Single)"...
Done.
Searching for "The Chipmunk Song"...
Done.
Searching for "Fever (Single)"...
Done.
Searching for "Mack The Knife"...
Done.
Searching for "The Three Bells"...
Done.
Searching for "Like Young (Single)"...
Done.
Searching for "High Hopes (Single)"...
Done.
Searching for "A Fool Such As I (Single)"...
Done.
Searching for "The Theme From A Summer Place"...
Done.
Searching for "Nice 'N Easy (Single)"...
Done.
Searching for "Mack The Knife (Single)"...
Done.
Searching for "Georgia On My Mind (Single)"...
Done.
Searching for "Are You Lonesome Tonight? (Single)"...
Done.
Searching for "Moon River"...
Done.
Searching for "Up A Lazy River"...
Done.
Searching for "The Second Time Around (Single)"...
Done.
Searching for "Take Five"...
Done.
Searching for "Big Bad John (Single)"...
Done.
Searching for "I Left My Heart In San Francisco"...
Done.
Searching

In [40]:
merged_df = pd.merge(df_grammys, df_lyrics, left_on='title', right_index=True, how='left')


In [41]:
merged_df.dropna(inplace=True)

In [42]:
merged_df.to_csv('genius_grammy.csv', index=False)

In [43]:
def read_lexicon(filename):
    word_sentiments = {}

    # Parse lexicon file
    with open(filename, 'r') as f:
        for line in f:
            # Ignore blank lines
            if line.strip():
                word, sentiment, value = line.split()
                if value == '1':
                    # Add sentiment to dictionary
                    cur_list = word_sentiments.get(word, [])
                    cur_list.append(sentiment)
                    word_sentiments[word] = cur_list

    return word_sentiments

In [44]:
def get_sentiment_percentages(track_df, sentiment):
    sentiment_pcts = []

    # Parse tracks
    for title in track_df.index.values:
        num_sentiment_words = 0
        all_words = str(track_df.loc[title]['lyrics']).split()

        # Parse words in lyrics
        for word in all_words:
            if sentiment in word_sentiments_dict.get(word, []):
                num_sentiment_words += 1

        sentiment_pcts.append(num_sentiment_words/len(all_words))

    return pd.DataFrame(data=sentiment_pcts, index=track_df.index.values, columns=[sentiment + '_pct'])


In [46]:
# Initialize sentiment dictionary and track dataframe
#word_sentiments_dict = read_lexicon('NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')


# Generate sentiment percentages based on NRC Lexicon sentiments
#sentiments = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']
#for sentiment in sentiments:
    #sentiment_df = get_sentiment_percentages(merged_df, sentiment)
    #merged_df = merged_df.join(sentiment_df)


# Calculate lyrical density and gloom index of tracks
#merged_df['lyrical_density'] = merged_df.apply(lambda row: len(row.lyrics.split()) / row.duration_ms * 1000, axis=1)
#merged_df['gloom_index'] = merged_df.apply(lambda row: 1 - (row.valence) + row.sadness_pct * (row.lyrical_density+1), axis=1)


# Rescale track_df such that gloom indices are in range [0, 100]
#rescale_df = merged_df
#max_gloom_value = merged_df['gloom_index'].max()
#min_gloom_value = merged_df['gloom_index'].min()
#rescale_df['gloom_index'] = (merged_df['gloom_index'] - min_gloom_value) / (max_gloom_value - min_gloom_value) * 100

#rescale_df['gloom_index'].sort_values(ascending=False)